## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-13-10-53-15 +0000,bbc,"Welsh Labour politician dies suddenly, aged 47",https://www.bbc.com/news/articles/cq58j18w32no...
1,2025-08-13-10-48-14 +0000,bbc,Former presiding officer Sir George Reid dies ...,https://www.bbc.com/news/articles/cvg3nj1g67lo...
2,2025-08-13-10-48-02 +0000,bbc,Average mortgage rates below 5% for first time...,https://www.bbc.com/news/articles/c4gzv41kw3jo...
3,2025-08-13-10-47-47 +0000,nypost,DC Mayor Muriel Bowser backtracks on support f...,https://nypost.com/2025/08/13/us-news/dc-mayor...
4,2025-08-13-10-46-55 +0000,nyt,Israel Hasn’t Prosecuted a Single Suspect for ...,https://www.nytimes.com/2025/08/13/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,70
221,putin,16
42,new,12
241,tariffs,11
41,china,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
30,2025-08-13-09-55-05 +0000,wapo,"Europe, Zelensky push to sway Trump before Ala...",https://www.washingtonpost.com/world/2025/08/1...,129
27,2025-08-13-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Duffy on Buttigieg’...,https://nypost.com/2025/08/13/us-news/ny-postc...,129
31,2025-08-13-09-47-42 +0000,nyt,Trump Will Discuss Ukraine With European Leade...,https://www.nytimes.com/2025/08/13/world/europ...,124
132,2025-08-12-22-53-38 +0000,nypost,What to expect from Friday’s Trump-Putin summi...,https://nypost.com/2025/08/12/world-news/what-...,121
214,2025-08-12-18-07-32 +0000,nypost,White House confirms Trump-Putin summit in Anc...,https://nypost.com/2025/08/12/us-news/white-ho...,120


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
30,129,2025-08-13-09-55-05 +0000,wapo,"Europe, Zelensky push to sway Trump before Ala...",https://www.washingtonpost.com/world/2025/08/1...
20,58,2025-08-13-10-00-00 +0000,nypost,DOT chief Sean Duffy charges predecessor Pete ...,https://nypost.com/2025/08/13/us-news/dot-chie...
283,58,2025-08-12-12-00-00 +0000,nypost,Ex-NYPD cop claims Bill de Blasio ‘didn’t like...,https://nypost.com/2025/08/12/us-news/ex-nypd-...
193,55,2025-08-12-19-21-00 +0000,wsj,The State Department released a long-delayed h...,https://www.wsj.com/politics/policy/state-depa...
100,50,2025-08-13-01-44-11 +0000,latimes,"She was killed while walking to her car, famil...",https://www.latimes.com/california/story/2025-...
102,49,2025-08-13-01-37-14 +0000,nypost,"Left-wing dark-money megadonors, including Geo...",https://nypost.com/2025/08/12/us-news/dark-mon...
224,47,2025-08-12-17-33-00 +0000,wsj,Crypto Entrepreneur Do Kwon Pleads Guilty to F...,https://www.wsj.com/finance/currencies/crypto-...
145,45,2025-08-12-22-09-00 +0000,wsj,"Year-over-year inflation held steady in July, ...",https://www.wsj.com/economy/cpi-inflation-july...
196,41,2025-08-12-19-08-00 +0000,wsj,The White House plans to conduct a far-reachin...,https://www.wsj.com/politics/policy/white-hous...
233,40,2025-08-12-16-45-08 +0000,nyt,Philippines Condemns China After South China S...,https://www.nytimes.com/2025/08/12/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
